#### 1. zookeeper有哪些使用场景?
* **分布式协调 (zk做回调)**  
  分布式服务中, 系统A发送一个请求到mq, 系统B从mq里接收消息后处理. 那系统A如何知道系统B处理完毕了呢? 一个可行的做法是:   
  系统A发送完消息后, 注册一个zk节点; 系统B收到消息并处理后修改zk节点上的值, 此时系统A会立刻收到通知  
  
* **分布式锁**  
  假设有2个系统同时接收到了一个请求, 但只有其中的一个系统可以执行该请求, 如何进行排它锁呢?   
  收到请求的系统可以在zk上创建一个临时节点, (创建临时节点主要是考虑到避免获得锁的系统因为宕机无法释放锁而产生死锁)
    * 先收到请求的系统会创建成功, 表示获得了锁; 
    * 后收到请求的系统会创建失败, 表示锁获取失败, 此后在这个zk临时节点上建立一个监听器, 一旦发现该节点被删除(有某个系统释放了锁), 就再次请求创建节点(获取锁). 如果是redis实现分布式锁, 获取锁失败的系统要每隔1s取轮训`setnx key 值 `
  
* **元数据配置**   
  因为元数据要求高可用和一致性, 所以用zk实现这些特点   
  
* **HA选主**   
  可以动态选主, 实现准备master之间的自动切换  

### 2. [重要]分布式锁是啥?  

#### 2.1 redis中有什么分布式锁实现
1. 最普通的实现方法  
  在redis中创建一个key表示锁`set my:lock 随机值 NX PX 30000`: 创建1个my:lock, 如果不存在就创建成功, 该key在30秒后失效.   
  只有创建成功的系统才表示获取锁成功, 该系统在处理完业务逻辑后, 使用lua脚本删除`my:lock`
    1. 为啥用随机值?   
      因为如果某个客户端获取了锁, 但是因为执行时间很长, 锁可能在redis中已过期, 于是其它系统获取锁; 如果不加判断就删除锁可能会误删其他系统获得的锁
    2. 该做法的一个问题是: 如果是采用一主多从的架构, master宕机后, 新的slave提升成master时可能并不包含锁, 导致其他系统也获得了锁;
    ```lua
        if redis.call(get,"my:lock"=="预期值") then
          return redis.call("del","my:lock")
        else
          return 0
        end
    ```
2. RedLock算法  
 该算法假设在redis-cluster环境中, 假设有5个master, 通过如下步骤获得一个锁  
    1. 获取当前时间戳(单位毫秒)
    2. 轮训在每隔master上创建锁
    3. 如果超过一半的master都建锁成功, 则该系统获得了锁;  
      否则, 依次删除建立好的锁
    4. 只要获取锁失败, 系统就要不断`轮训->建立->判断成功建立的是否大于一半->删除` 
    
#### 2.2 zookeeper实现分布式锁
1. 基于临时节点实现  
收到请求的系统可以在zk上创建一个临时节点, (创建临时节点主要是考虑到避免获得锁的系统因为宕机无法释放锁而产生死锁)
    * 先收到请求的系统会创建成功, 表示获得了锁; 
    * 后收到请求的系统会创建失败, 表示锁获取失败, 此后在这个zk临时节点上建立一个监听器, 一旦发现该节点被删除(有某个系统释放了锁), 就再次请求创建节点(获取锁). 如果是redis实现分布式锁, 获取锁失败的系统要每隔1s取轮训`setnx key 值 `
    
2. 基于临时顺序节点  